In [83]:
from tensorflow.keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Input, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.layers import Conv2D
from keras import backend as k
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import pickle
import tensorflow as tf

In [84]:
def loadPickleData(file):
    data = pickle.load(open(file, 'rb'));
    images = data[0];
    labels = data[1];
    return [images, labels]

In [85]:
[images,labels] = loadPickleData("./../Data/pickledFiles/Num_SingleDigit_3channel.p")

In [86]:
y = np_utils.to_categorical(labels);
print(y);
print(y.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
(2000, 10)


In [87]:
X = images
X.shape
# X = np.reshape(X, (2000,60,40,1))

(2000, 60, 40, 3)

In [88]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2);

In [89]:
model = applications.VGG19(weights = "imagenet", include_top=False,\
                           input_shape = (60, 40, 3))

In [91]:
print(len(model.layers))
for layer in model.layers[0:18]:
    layer.trainable = False

22


In [92]:
inputt = Input(shape =  (60,40,3), name = 'captcha_input')
output_vgg16_conv = model(inputt);

In [93]:
x = Flatten(name='flatten')(output_vgg16_conv)


In [94]:
predictions = Dense(10, activation="softmax")(x)


In [95]:
model_final = Model(inputt,predictions)


In [96]:
model_final.compile(loss = "categorical_crossentropy",\
                    optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9), metrics=["accuracy"])

In [97]:
print(model_final.summary())

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
captcha_input (InputLayer)   [(None, 60, 40, 3)]       0         
_________________________________________________________________
vgg19 (Functional)           (None, 1, 1, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                5130      
Total params: 20,029,514
Trainable params: 4,724,746
Non-trainable params: 15,304,768
_________________________________________________________________
None


In [98]:
lossCheck = ModelCheckpoint('singleCharLoss.h5', monitor='val_loss', verbose=True, save_best_only=True)
accCheck = ModelCheckpoint('singleCharAcc.h5', monitor='val_accuracy', verbose=True, save_best_only=True)

In [99]:
history = model_final.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=300, epochs=150, verbose=1, callbacks=[lossCheck, accCheck])

Epoch 1/150
6/6 [==============================] - 8s 1s/step - loss: 11.8442 - accuracy: 0.1112 - val_loss: 5.3435 - val_accuracy: 0.1350

Epoch 00001: val_loss improved from inf to 5.34347, saving model to singleCharLoss.h5

Epoch 00001: val_accuracy improved from -inf to 0.13500, saving model to singleCharAcc.h5
Epoch 2/150
6/6 [==============================] - 8s 1s/step - loss: 3.4302 - accuracy: 0.1519 - val_loss: 2.5769 - val_accuracy: 0.1200

Epoch 00002: val_loss improved from 5.34347 to 2.57685, saving model to singleCharLoss.h5

Epoch 00002: val_accuracy did not improve from 0.13500
Epoch 3/150
6/6 [==============================] - 8s 1s/step - loss: 2.3367 - accuracy: 0.1406 - val_loss: 2.3363 - val_accuracy: 0.1150

Epoch 00003: val_loss improved from 2.57685 to 2.33631, saving model to singleCharLoss.h5

Epoch 00003: val_accuracy did not improve from 0.13500
Epoch 4/150
6/6 [==============================] - 8s 1s/step - loss: 2.2697 - accuracy: 0.1300 - val_loss: 2.296

6/6 [==============================] - 8s 1s/step - loss: 1.9658 - accuracy: 0.2375 - val_loss: 2.1335 - val_accuracy: 0.1825

Epoch 00030: val_loss improved from 2.14285 to 2.13348, saving model to singleCharLoss.h5

Epoch 00030: val_accuracy improved from 0.16500 to 0.18250, saving model to singleCharAcc.h5
Epoch 31/150
6/6 [==============================] - 8s 1s/step - loss: 1.9506 - accuracy: 0.2506 - val_loss: 2.1196 - val_accuracy: 0.1700

Epoch 00031: val_loss improved from 2.13348 to 2.11964, saving model to singleCharLoss.h5

Epoch 00031: val_accuracy did not improve from 0.18250
Epoch 32/150
6/6 [==============================] - 8s 1s/step - loss: 1.9330 - accuracy: 0.2512 - val_loss: 2.1233 - val_accuracy: 0.1750

Epoch 00032: val_loss did not improve from 2.11964

Epoch 00032: val_accuracy did not improve from 0.18250
Epoch 33/150
6/6 [==============================] - 8s 1s/step - loss: 1.9239 - accuracy: 0.2575 - val_loss: 2.1097 - val_accuracy: 0.1650

Epoch 00033: val

Epoch 61/150
6/6 [==============================] - 9s 1s/step - loss: 1.6156 - accuracy: 0.3938 - val_loss: 2.0843 - val_accuracy: 0.2475

Epoch 00061: val_loss improved from 2.08651 to 2.08426, saving model to singleCharLoss.h5

Epoch 00061: val_accuracy did not improve from 0.27250
Epoch 62/150
6/6 [==============================] - 9s 1s/step - loss: 1.6094 - accuracy: 0.3938 - val_loss: 2.0789 - val_accuracy: 0.2700

Epoch 00062: val_loss improved from 2.08426 to 2.07894, saving model to singleCharLoss.h5

Epoch 00062: val_accuracy did not improve from 0.27250
Epoch 63/150
6/6 [==============================] - 9s 1s/step - loss: 1.5806 - accuracy: 0.4275 - val_loss: 2.0686 - val_accuracy: 0.2725

Epoch 00063: val_loss improved from 2.07894 to 2.06859, saving model to singleCharLoss.h5

Epoch 00063: val_accuracy did not improve from 0.27250
Epoch 64/150
6/6 [==============================] - 9s 1s/step - loss: 1.5590 - accuracy: 0.4263 - val_loss: 2.0761 - val_accuracy: 0.2725

Ep

6/6 [==============================] - 9s 1s/step - loss: 1.2169 - accuracy: 0.5869 - val_loss: 1.9596 - val_accuracy: 0.3375

Epoch 00091: val_loss did not improve from 1.92836

Epoch 00091: val_accuracy did not improve from 0.34500
Epoch 92/150
6/6 [==============================] - 9s 1s/step - loss: 1.2047 - accuracy: 0.5856 - val_loss: 1.9266 - val_accuracy: 0.3575

Epoch 00092: val_loss improved from 1.92836 to 1.92658, saving model to singleCharLoss.h5

Epoch 00092: val_accuracy improved from 0.34500 to 0.35750, saving model to singleCharAcc.h5
Epoch 93/150
6/6 [==============================] - 9s 1s/step - loss: 1.1731 - accuracy: 0.6006 - val_loss: 1.9271 - val_accuracy: 0.3450

Epoch 00093: val_loss did not improve from 1.92658

Epoch 00093: val_accuracy did not improve from 0.35750
Epoch 94/150
6/6 [==============================] - 9s 1s/step - loss: 1.1614 - accuracy: 0.6050 - val_loss: 1.9294 - val_accuracy: 0.3650

Epoch 00094: val_loss did not improve from 1.92658

Epo

6/6 [==============================] - 9s 1s/step - loss: 0.9368 - accuracy: 0.6938 - val_loss: 1.9119 - val_accuracy: 0.3850

Epoch 00122: val_loss did not improve from 1.89404

Epoch 00122: val_accuracy did not improve from 0.40000
Epoch 123/150
6/6 [==============================] - 9s 1s/step - loss: 0.9362 - accuracy: 0.6938 - val_loss: 1.9221 - val_accuracy: 0.3900

Epoch 00123: val_loss did not improve from 1.89404

Epoch 00123: val_accuracy did not improve from 0.40000
Epoch 124/150
6/6 [==============================] - 14s 2s/step - loss: 0.9170 - accuracy: 0.6975 - val_loss: 1.9376 - val_accuracy: 0.3625

Epoch 00124: val_loss did not improve from 1.89404

Epoch 00124: val_accuracy did not improve from 0.40000
Epoch 125/150
6/6 [==============================] - 15s 3s/step - loss: 0.9055 - accuracy: 0.7031 - val_loss: 1.9142 - val_accuracy: 0.3775

Epoch 00125: val_loss did not improve from 1.89404

Epoch 00125: val_accuracy did not improve from 0.40000
Epoch 126/150
6/6 [=

In [56]:
history1 = model_final.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=300, epochs=50, verbose=1);

Epoch 1/50
6/6 [==============================] - 8s 1s/step - loss: 1.3719 - accuracy: 0.4825 - val_loss: 1.7787 - val_accuracy: 0.3750
Epoch 2/50
6/6 [==============================] - 8s 1s/step - loss: 1.3715 - accuracy: 0.4894 - val_loss: 1.7508 - val_accuracy: 0.3750
Epoch 3/50
6/6 [==============================] - 8s 1s/step - loss: 1.3429 - accuracy: 0.4963 - val_loss: 1.7647 - val_accuracy: 0.4050
Epoch 4/50
6/6 [==============================] - 8s 1s/step - loss: 1.3140 - accuracy: 0.5256 - val_loss: 1.7480 - val_accuracy: 0.4050
Epoch 5/50
6/6 [==============================] - 8s 1s/step - loss: 1.2916 - accuracy: 0.5263 - val_loss: 1.7367 - val_accuracy: 0.4025
Epoch 6/50
6/6 [==============================] - 8s 1s/step - loss: 1.2643 - accuracy: 0.5375 - val_loss: 1.7536 - val_accuracy: 0.3775
Epoch 7/50
6/6 [==============================] - 8s 1s/step - loss: 1.2572 - accuracy: 0.5462 - val_loss: 1.7362 - val_accuracy: 0.4150
Epoch 8/50
6/6 [=========================

In [57]:
history2 = model_final.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=300, epochs=50, verbose=1);

Epoch 1/50
6/6 [==============================] - 8s 1s/step - loss: 0.7408 - accuracy: 0.7550 - val_loss: 1.7918 - val_accuracy: 0.4675
Epoch 2/50
6/6 [==============================] - 8s 1s/step - loss: 0.7422 - accuracy: 0.7575 - val_loss: 1.8043 - val_accuracy: 0.4625
Epoch 3/50
6/6 [==============================] - 8s 1s/step - loss: 0.7343 - accuracy: 0.7569 - val_loss: 1.8027 - val_accuracy: 0.4650
Epoch 4/50
6/6 [==============================] - 8s 1s/step - loss: 0.7151 - accuracy: 0.7619 - val_loss: 1.7949 - val_accuracy: 0.4775
Epoch 5/50
6/6 [==============================] - 8s 1s/step - loss: 0.7033 - accuracy: 0.7713 - val_loss: 1.8078 - val_accuracy: 0.4525
Epoch 6/50
6/6 [==============================] - 8s 1s/step - loss: 0.6999 - accuracy: 0.7706 - val_loss: 1.7969 - val_accuracy: 0.4775
Epoch 7/50
6/6 [==============================] - 8s 1s/step - loss: 0.6904 - accuracy: 0.7756 - val_loss: 1.8212 - val_accuracy: 0.4575
Epoch 8/50
6/6 [=========================

In [61]:
max(max(history.history["val_accuracy"]), max(history1.history["val_accuracy"]), max(history2.history["val_accuracy"]))

0.47999998927116394

In [63]:
xtr = model_final.predict(X_train, batch_size = 200, verbose=1)

8/8 [==============================] - 5s 569ms/step


In [64]:
Xtest = model_final.predict(X_test, batch_size = 100, verbose=1);

4/4 [==============================] - 1s 294ms/step


In [65]:
print(xtr.shape)

(1600, 10)


In [66]:
resize = np.prod(xtr.shape[1:])

In [67]:
xtr = np.reshape(xtr, (1600, resize));
Xtest = np.reshape(Xtest, (400, resize));

In [74]:
bf_train_s = xtr
bf_test_s = Xtest

In [75]:
y_train_mc = np.argmax(y_train, axis=1)
y_test_mc = np.argmax(y_test, axis = 1)

In [80]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm


In [77]:
clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', verbose=1,max_iter=10000)

In [78]:
clf.fit(bf_train_s,y_train_mc)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


LogisticRegression(max_iter=10000, multi_class='multinomial', verbose=1)

In [79]:
print(clf.score(bf_test_s, y_test_mc))

0.4675


In [81]:
multiClassSVM = svm.LinearSVC(loss = 'squared_hinge', penalty = 'l2',C=1/5000 ,max_iter=800000,verbose=1) 
multiClassSVM.fit(bf_train_s,y_train_mc)

[LibLinear]

LinearSVC(C=0.0002, max_iter=800000, verbose=1)

In [82]:
multiClassSVM.score(bf_test_s, y_test_mc)

0.465